# Credit Card Transactions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist, pdist
from sklearn.cluster import KMeans
%matplotlib inline

In [2]:
df1= pd.read_csv('cc_info.csv')
df2 = pd.read_csv('transactions.csv', parse_dates= ['date'])


In [3]:
df1.head()

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [4]:
df2.head()
df2['date'].dtype

dtype('<M8[ns]')

In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294588 entries, 0 to 294587
Data columns (total 5 columns):
credit_card                  294588 non-null int64
date                         294588 non-null datetime64[ns]
transaction_dollar_amount    294588 non-null float64
Long                         294588 non-null float64
Lat                          294588 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 11.2 MB


In [6]:
df3 =pd.merge(df1, df2, on= 'credit_card', how = 'left')
df3.head()

,credit_card,city,state,zipcode,credit_card_limit,date,transaction_dollar_amount,Long,Lat
0,1280981422329509,Dallas,PA,18612,6000,2015-08-05 00:59:19,11.94,-75.964527,41.353578
1,1280981422329509,Dallas,PA,18612,6000,2015-10-29 18:23:04,5.76,-76.019703,41.311467
2,1280981422329509,Dallas,PA,18612,6000,2015-10-25 17:50:48,25.84,-76.043031,41.291053
3,1280981422329509,Dallas,PA,18612,6000,2015-09-05 17:39:43,68.89,-75.944299,41.327282
4,1280981422329509,Dallas,PA,18612,6000,2015-09-04 14:12:59,27.01,-75.997259,41.352099


In [7]:
#df3['date'] = pd.DatetimeIndex(df3['date']).month
#df3['date']
df3['date'] = pd.to_datetime(df3['date'])

In [8]:
df3['date'].dtype

dtype('<M8[ns]')

In [9]:
df3.sort_values(by = ['date'], inplace = True)
df3.head()

,credit_card,city,state,zipcode,credit_card_limit,date,transaction_dollar_amount,Long,Lat
179840,9484591448272784,New Orleans,LA,70112,4000,2015-07-31 09:39:48,17.99,-90.045639,29.889039
240501,7053196367895112,New York,NY,10001,18000,2015-07-31 11:03:48,12.09,-74.027561,40.689615
255316,9528285469413252,Washington,NH,3280,40000,2015-07-31 11:10:14,78.21,-72.139485,43.108100
261083,1845720274833905,El Paso,IL,61738,20000,2015-07-31 11:28:55,74.41,-89.002148,40.804323
148163,7850942767136368,Washington,NH,3280,4000,2015-07-31 11:38:51,54.89,-72.025675,43.210753


In [10]:
df3['month'] = df3['date'].apply(lambda x: x.month)
df3['day'] = df3['date'].apply(lambda x: x.day)
df3['weekday'] = df3['date'].apply(lambda x: x.dayofweek)
df3['hour'] = df3['date'].apply(lambda x: x.hour)
df3.drop(['date'], axis = 1, inplace = True)

In [11]:
#df3['date'].resample('M', convention= 'start').asfreq()
df3.head()

,credit_card,city,state,zipcode,credit_card_limit,transaction_dollar_amount,Long,Lat,month,day,weekday,hour
179840,9484591448272784,New Orleans,LA,70112,4000,17.99,-90.045639,29.889039,7,31,4,9
240501,7053196367895112,New York,NY,10001,18000,12.09,-74.027561,40.689615,7,31,4,11
255316,9528285469413252,Washington,NH,3280,40000,78.21,-72.139485,43.108100,7,31,4,11
261083,1845720274833905,El Paso,IL,61738,20000,74.41,-89.002148,40.804323,7,31,4,11
148163,7850942767136368,Washington,NH,3280,4000,54.89,-72.025675,43.210753,7,31,4,11


In [12]:
def monthly_transactions(df):
    total_trans = df['transaction_dollar_amount'].sum()
    credit_limit = df['credit_card_limit'].values[0]
    return pd.Series([total_trans, credit_limit], index = ['total_transaction','credit_limit'])

In [13]:
#df3.groupby(['credit_card','month']).describe()
grouped = df3.groupby(['credit_card','month']).apply(monthly_transactions)
grouped = grouped.reset_index()
#grouped

In [14]:
lis_num = df3.groupby(['credit_card','month','credit_card_limit'])['transaction_dollar_amount'].agg('sum')
lis_num= pd.DataFrame(lis_num)
lis_num= lis_num.reset_index()
lis_num.head()


,credit_card,month,credit_card_limit,transaction_dollar_amount
0,1003715054175576,7,20000,162.56
1,1003715054175576,8,20000,8441.51
2,1003715054175576,9,20000,11281.56
3,1003715054175576,10,20000,8954.21
4,1013870087888817,7,15000,281.43


In [15]:
#### Calculate the credit cards which haven't exceed credit card limit
credit_count = lis_num.loc[lis_num['credit_card_limit']>=lis_num['transaction_dollar_amount'], 'credit_card' ]
#cred_num = credit_count.unique()
#credit_count
cred_num = pd.DataFrame(credit_count.groupby(credit_count).count().sort_values(ascending = False))
 #= cred_num.reset_index()

cred_num.columns = ['count_number'] 
cred_num= cred_num.reset_index()
list0 =cred_num.loc[cred_num['count_number']==4, 'credit_card']
print('\nnumber of credit cards which havent exceed limit:',len(list0))
list0.head()


number of credit cards which havent exceed limit: 751


0    1003715054175576
1    4546345297067747
2    5177967593503491
3    5175507030601876
4    5171595359553508
Name: credit_card, dtype: int64

In [16]:
df3.head()

,credit_card,city,state,zipcode,credit_card_limit,transaction_dollar_amount,Long,Lat,month,day,weekday,hour
179840,9484591448272784,New Orleans,LA,70112,4000,17.99,-90.045639,29.889039,7,31,4,9
240501,7053196367895112,New York,NY,10001,18000,12.09,-74.027561,40.689615,7,31,4,11
255316,9528285469413252,Washington,NH,3280,40000,78.21,-72.139485,43.108100,7,31,4,11
261083,1845720274833905,El Paso,IL,61738,20000,74.41,-89.002148,40.804323,7,31,4,11
148163,7850942767136368,Washington,NH,3280,4000,54.89,-72.025675,43.210753,7,31,4,11


In [17]:
df3['credit_card_limit'].values[0]

4000

In [18]:
def daily_surpass_limit(df, month, day):
   #df['credit_card']
    df0 = df.loc[(df['month']== month) & (df['day'] <=day),['credit_card','credit_card_limit','transaction_dollar_amount']]
    
    def sum_credit_card(dfn):
        
        return pd.Series([dfn['transaction_dollar_amount'].sum(), dfn['credit_card_limit'].values[0]],index= ['transction sum','limit'] )
    
    
    
    grouped2= df0.groupby(['credit_card']).apply(sum_credit_card)
    grouped2 = grouped2.reset_index()
    
    #df.loc[df['month']== month & df['day'] <=day,'transaction_dollar_amount'].sum()


    return grouped2[grouped2['transction sum']>grouped2['limit']]


daily_surpass_limit(df3, 8, 20)

,credit_card,transction sum,limit
194,2850146878241916,4012.81,4000.0
319,3936887050542904,5352.07,4000.0
341,4118286032166087,3416.52,3000.0
383,4462290711836916,3119.53,2000.0
525,5723635641134781,2161.30,2000.0
698,7299183791723634,7074.49,7000.0
772,7850942767136368,5704.00,4000.0
834,8522875529951473,5860.89,5000.0
874,8972201384562696,5266.08,5000.0


In [19]:
df3.head()

,credit_card,city,state,zipcode,credit_card_limit,transaction_dollar_amount,Long,Lat,month,day,weekday,hour
179840,9484591448272784,New Orleans,LA,70112,4000,17.99,-90.045639,29.889039,7,31,4,9
240501,7053196367895112,New York,NY,10001,18000,12.09,-74.027561,40.689615,7,31,4,11
255316,9528285469413252,Washington,NH,3280,40000,78.21,-72.139485,43.108100,7,31,4,11
261083,1845720274833905,El Paso,IL,61738,20000,74.41,-89.002148,40.804323,7,31,4,11
148163,7850942767136368,Washington,NH,3280,4000,54.89,-72.025675,43.210753,7,31,4,11


## Question 3
implement an unsupervised algorithm returning all transactions seem unusual

In [29]:
df4= df3
df3.head()

,credit_card,city,state,zipcode,credit_card_limit,transaction_dollar_amount,Long,Lat,month,day,weekday,hour
credit_card,,,,,,,,,,,,
9484591448272784,9484591448272784,New Orleans,LA,70112,4000,17.99,-90.045639,29.889039,7,31,4,9
7053196367895112,7053196367895112,New York,NY,10001,18000,12.09,-74.027561,40.689615,7,31,4,11
9528285469413252,9528285469413252,Washington,NH,3280,40000,78.21,-72.139485,43.108100,7,31,4,11
1845720274833905,1845720274833905,El Paso,IL,61738,20000,74.41,-89.002148,40.804323,7,31,4,11
7850942767136368,7850942767136368,Washington,NH,3280,4000,54.89,-72.025675,43.210753,7,31,4,11


There are two ways to check whether a  transaction is unusual or not:
1. check the transaction dollar amount to the mean value, if it deviates a lot, then it could be a fraud
2. check the transaction place if it is far away from the usual places, then it is possible that it is a fraud. 
Since time is limited here, we will only talk about the transaction dollar amount case.

In [43]:
def percent4(df):
    ps= [25,50,75]
    return pd.Series(np.percentile(df,ps),index = ['{}%'.format(p) for p in ps])
    


df3_stats= df3.groupby(['credit_card'])['transaction_dollar_amount'].apply(percent4).unstack()
df3_stats.head()

C:\Users\Jason\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 'credit_card' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  import sys


,25%,50%,75%
credit_card,,,
1003715054175576,59.01,93.73,124.63
1013870087888817,58.09,87.67,128.02
1023820165155391,55.43,85.78,122.83
1073931538936472,108.08,147.19,194.83
1077622576192810,122.93,148.18,160.11


In [45]:
temp = pd.merge(df3, df3_stats, on = 'credit_card', how = 'left')
temp.head()

C:\Users\Jason\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: FutureWarning: 'credit_card' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


,credit_card,city,state,zipcode,credit_card_limit,transaction_dollar_amount,Long,Lat,month,day,weekday,hour,25%,50%,75%
0,9484591448272784,New Orleans,LA,70112,4000,17.99,-90.045639,29.889039,7,31,4,9,8.9975,12.410,16.9525
1,7053196367895112,New York,NY,10001,18000,12.09,-74.027561,40.689615,7,31,4,11,45.0200,70.940,97.5000
2,9528285469413252,Washington,NH,3280,40000,78.21,-72.139485,43.108100,7,31,4,11,82.2000,137.030,189.2300
3,1845720274833905,El Paso,IL,61738,20000,74.41,-89.002148,40.804323,7,31,4,11,37.6825,61.335,84.6125
4,7850942767136368,Washington,NH,3280,4000,54.89,-72.025675,43.210753,7,31,4,11,28.1400,45.850,64.2100


## Reduce dimension and visualize

In [47]:
temp0= temp.drop(['city','state','zipcode','Long','Lat','month','day','weekday','hour'], axis= 1)
temp0.head()

,credit_card,credit_card_limit,transaction_dollar_amount,25%,50%,75%
0,9484591448272784,4000,17.99,8.9975,12.410,16.9525
1,7053196367895112,18000,12.09,45.0200,70.940,97.5000
2,9528285469413252,40000,78.21,82.2000,137.030,189.2300
3,1845720274833905,20000,74.41,37.6825,61.335,84.6125
4,7850942767136368,4000,54.89,28.1400,45.850,64.2100


In [48]:
temp0.to_csv('percentile_tansaction.csv')

In [28]:
k_means_fit = KMeans(n_clusters =2, max_iter= 300)
k_means_fit.fit(dfm)
k_labels = k_means_fit.labels_

In [ ]:
k_labels